In [ ]:
import numpy as np
import healpy as hp
from pathlib import Path

In [ ]:
output_dir = Path("production-data") / "dust_gnilc"

In [ ]:
datadir = output_dir / "raw"

In [ ]:
output_nside = 2048

In [ ]:
output_lmax = int(min(2.5 * output_nside, 8192 * 2))

## Large scales

In [ ]:
alm_log_pol_tens_large_scale = hp.read_alm(
    datadir
    / "gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax3072_complex64.fits.gz",
    hdu=(1, 2, 3),
)

In [ ]:
map_log_pol_tens_large_scale = hp.alm2map(
    alm_log_pol_tens_large_scale.astype(np.complex128), nside=output_nside
)

In [ ]:
map_log_pol_tens_large_scale

## Galactic mask

In [ ]:
galactic_mask = (
    hp.ud_grade(
        hp.read_map(datadir / "HFI_Mask_GalPlane-apo2_2048_R2.00_GAL080_noapo.fits.gz"),
        output_nside,
    )
    == 1
)

## Small scales modulation

In [ ]:
modulate_alm = {
    k: hp.read_alm(datadir / f"gnilc_dust_{k}_modulation_alms_lmax3072.fits.gz").astype(
        np.complex128
    )
    for k in ["temperature", "polarization"]
}

## Small scales

In [ ]:
cl_small_scale = hp.read_cl(
    datadir / "gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz"
)

In [ ]:
synalm_lmax = 8192 * 2  # it needs to be the same for all output nside
# synalm_lmax = output_lmax
np.random.seed(8192)

alm_log_pol_tens_small_scale = hp.synalm(
    list(cl_small_scale),
    lmax=synalm_lmax,
    new=True,
)

In [ ]:
alm_log_pol_tens_small_scale = [
    hp.almxfl(each, np.ones(output_lmax))
    for each in alm_log_pol_tens_small_scale
]
map_log_pol_tens_small_scale = hp.alm2map(
    alm_log_pol_tens_small_scale, nside=output_nside
)

In [ ]:
map_log_pol_tens_small_scale

In [ ]:
map_log_pol_tens_small_scale[0][galactic_mask] *= hp.alm2map(
    modulate_alm["temperature"], output_nside
)[galactic_mask]
map_log_pol_tens_small_scale[1:, galactic_mask] *= hp.alm2map(
    modulate_alm["polarization"], output_nside
)[galactic_mask]
assert np.isnan(map_log_pol_tens_small_scale).sum() == 0

In [ ]:
map_log_pol_tens_small_scale

## Combine scales

* Combine small and large scale maps
* Transform from logpoltens to IQU
* Write output map

In [ ]:
map_log_pol_tens = map_log_pol_tens_large_scale
map_log_pol_tens += map_log_pol_tens_small_scale

In [ ]:
del map_log_pol_tens_small_scale

In [ ]:
from pysm3.utils import log_pol_tens_to_map

In [ ]:
output_map = log_pol_tens_to_map(map_log_pol_tens)

In [ ]:
output_map

In [ ]:
del map_log_pol_tens

## Galactic plane fix

In [ ]:
galplane_fix = hp.read_map(datadir / "gnilc_dust_galplane.fits.gz", (0, 1, 2, 3))

In [ ]:
output_map *= hp.ud_grade(galplane_fix[3], output_nside)
output_map += hp.ud_grade(galplane_fix[:3] * (1 - galplane_fix[3]), output_nside)

In [ ]:
output_map

## Color correction

Planck 353 GHz color correction https://github.com/galsci/pysm/issues/99

In [ ]:
output_map *= 0.911

In [ ]:
output_map

In [ ]:
hp.write_map(
    output_dir / f"gnilc_dust_template_nside{output_nside}.fits",
    output_map,
    dtype=np.float32,
    overwrite=True,
    column_units = "uK_RJ",
    extra_header = [("lmax", output_lmax), ("ref_freq", "353 GHz")]
)